# ###주의 이 파일은 worker, master를 연결만 하는 스크립트 입니다.
# worker, master에 대한 변경이 필요하면 terraform, userdata를 고쳐야 합니다.

# 순서!

## 1. terraform으로 클러스터 생성
## 2. user data로 들어가는 스크립트 파일로 kubernetes 설치
## 3. 설치 후 ending text 를 echo함
## 4. kube_join.txt 파일의 값을 읽어서 worker에 뿌림
## 5. 다 작성하면 쉘스크립트에 테라폼 apply 까지 묶어서 한번에 가자

# 정말 마지막 테스트 worker 3개.
## 아... 그냥 fully managed service 써도 되는 개발자들 부럽다...

# 근데 테라폼 한번 배워서 진짜 요긴하게 잘 써먹네.
## 솔직히 테라폼 없었으면 에러 한두번 발생했을때 인스턴스 다시 키는거 너무 힘들어서 좌절했을듯

https://askubuntu.com/questions/983351/how-to-install-terraform-in-ubuntu

!apt install -y wget
!wget https://releases.hashicorp.com/terraform/1.0.7/terraform_1.0.7_linux_amd64.zip
!unzip terraform_1.0.7_linux_amd64.zip
!mv terraform /usr/local/bin/
!terraform --version 

!rm terraform_1.0.7_linux_amd64.zip

In [1]:
#!terraform init

In [2]:
#!terraform destroy -auto-approve

In [3]:
#!terraform plan

In [4]:
!terraform apply -auto-approve

aws_launch_configuration.kube_worker_lc (deposed object 39f8ade0): Refreshing state... [id=terraform-20230520144800625800000001]
aws_vpc.kube_vpc: Refreshing state... [id=vpc-0b4d57440f292db0e]
aws_launch_configuration.kube_worker_lc (deposed object 443eae75): Refreshing state... [id=terraform-20230520144516615300000001]
aws_route_table.public_rt: Refreshing state... [id=rtb-0cc163def2f01f7db]
aws_internet_gateway.kube_igw: Refreshing state... [id=igw-0887a129754d7474b]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0ad1624c9d0fad1a3]
aws_security_group.sg_worker: Refreshing state... [id=sg-0dfd5b1a60abb644a]
aws_security_group.sg_master: Refreshing state... [id=sg-021225d4472562ad3]
aws_route.public_rt_igw: Refreshing state... [id=r-rtb-0cc163def2f01f7db1080289494]
aws_route_table_association.public_rt_association: Refreshing state... [id=rtbassoc-0e848ae8754506f0a]
aws_launch_configuration.kube_worker_lc: Refreshing state... [id=terraform-20230520145016762100000001]
aws_spo

aws_ec2_tag.master_tag: Destroying... [id=i-0b388ae62c207c7e1,Name]
aws_autoscaling_group.kube_worker_asg: Destroying... [id=terraform-20230520145017940700000002]
aws_ec2_tag.master_tag: Destruction complete after 1s
aws_spot_instance_request.kube_master: Destroying... [id=sir-67tegkfp]
aws_autoscaling_group.kube_worker_asg: Still destroying... [id=terraform-20230520145017940700000002, 10s elapsed]
aws_spot_instance_request.kube_master: Still destroying... [id=sir-67tegkfp, 10s elapsed]
aws_autoscaling_group.kube_worker_asg: Destruction complete after 20s
aws_launch_configuration.kube_worker_lc: Creating...
aws_spot_instance_request.kube_master: Still destroying... [id=sir-67tegkfp, 20s elapsed]
aws_launch_configuration.kube_worker_lc: Creation complete after 1s [id=terraform-20230520145223772600000001]
aws_autoscaling_group.kube_worker_asg: Creating...
aws_spot_instance_request.kube_master: Still destroying... [id=sir-67tegkfp, 30s elapsed]
aws_autoscaling_group.kube_worker_asg: Still

In [5]:
import time

print('wait for terraform')
time.sleep(30)

wait for terraform


## 만들어진 인스턴스 목록 획득

https://stackoverflow.com/questions/33297172/boto3-error-botocore-exceptions-nocredentialserror-unable-to-locate-credential

# 인스턴스 연결 설정 들

!mkdir -p ~/.aws
#!ls ~/.aws

!(echo " \
[default] \n \
aws_access_key_id = 키키키키 \n \
aws_secret_access_key = 쉬크릿쉬크릿쉬크릿쉬크릿 \
") | tee ~/.aws/credentials

!(echo " \
[default] \n \
region = us-east-1 \n \
output = json \
") | tee ~/.aws/config|

In [6]:
import boto3
import time

# 인스턴스가 생성되는 걸 기다렸다가 이름을 기준으로 master, worker 분류
## terraform에서는 현재 불가능 하거나, 두번에 나눠 해야 되서 여기서 진행

In [7]:
is_started = True

In [8]:
def get_cluster_instances():
    ec2 = boto3.resource('ec2')
    
    master_list = []
    worker_list = []
    
    for instance in ec2.instances.all():

        if instance.state['Name'] == 'pending' : 
            print("Wait for pending", instance )
            instance.wait_until_running()
            is_pending=True
            time.sleep(3)

        elif instance.state['Name'] == 'running' :
            for tag in instance.tags :
                if tag['Key'] == 'Name' and tag['Value'] == 'kube_master' :
                    print(instance, tag)
                    master_list.append(instance)
                if tag['Key'] == 'Name' and tag['Value'] == 'kube_worker' :
                    print(instance, tag)
                    worker_list.append(instance)

    return master_list, worker_list


## wait for pending이 있어서 사용했는데도
## 데이터가 처음 긁을땐 안가져와지는 현상이 있어 get again

In [9]:
master_list = []
worker_list = []
if is_started :
    print('wait for starting')
    time.sleep(10)
    is_started = False
    
master_list, worker_list = get_cluster_instances()
#이 코드 노드 숫자 늘어나면 분명히 문제 생길듯
#정 뭣하면 master 먼저 키고 다 된거 확인 한 후 worker 켜서 userdata에 넣어두던가...
if len(master_list) == 0 or len(worker_list) == 0 :
    print("get again")
    master_list, worker_list = get_cluster_instances()

wait for starting
ec2.Instance(id='i-0e1d21c04c3d97854') {'Key': 'Name', 'Value': 'kube_master'}
ec2.Instance(id='i-0780cdbb01b12ae6a') {'Key': 'Name', 'Value': 'kube_worker'}
ec2.Instance(id='i-0d046300b4951283c') {'Key': 'Name', 'Value': 'kube_worker'}


In [10]:
master_list

[ec2.Instance(id='i-0e1d21c04c3d97854')]

In [11]:
master_list[0].public_ip_address

'52.90.68.113'

In [12]:
worker_list[0].public_ip_address

'174.129.95.87'

In [13]:
worker_list

[ec2.Instance(id='i-0780cdbb01b12ae6a'),
 ec2.Instance(id='i-0d046300b4951283c')]

In [14]:
import botocore
import paramiko
import time

#전형적인 template pattern이 필요한 사례같아서 한번 써봄
def exec_process(instance_ip, command_list, func_list):

    key = paramiko.RSAKey.from_private_key_file('DevOps_Study.pem')
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    print(instance_ip)
    result_list = []
    try:
        client.connect(hostname=instance_ip, username="ubuntu", pkey=key)

        for i in range(len(func_list)):
            result_list.append(
                func_list[i](instance_ip,command_list[i],client)
            )

        client.close()
        
    except Exception as e:
        print(e)
        client.close()
    finally:
        return result_list

def exec_command_func(instance_ip, command, client):
    # Execute a command(cmd) after connecting/ssh to an instance
    stdin, stdout, stderr = client.exec_command(command)
    
    output = str(stdout.read() , 'utf-8')
    
    if output[-1] == "\n":
        output = output[:-1]
    print(output)
    #print(stderr)
    
    if output == "kubernetes_ready\n" : #"end\n":#
        print(instance_ip, "is ready")
    return output

def check_in_while_loop(instance_ip, command, client):
    while(True) :
        # Execute a command(cmd) after connecting/ssh to an instance
        stdin, stdout, stderr = client.exec_command(command)
        
        output = str(stdout.read() , 'utf-8')
        
        if output[-1] == "\n":
            output = output[:-1]
        
        print(output)
        #print(stderr)
        
        if output == "kubernetes_ready" : #"end\n":#
            print(master_ip, "is ready")
            return True
        time.sleep(10)
            
    return False

# user data가 다 끝날때까지 대기 후 node 연결 정보 수집

In [15]:
def try_until_finish(master_ip):
    try :
        result_list = exec_process(instance_ip=master_ip, 
                                       command_list=['sudo tail -n 1 /var/log/user-data.log',
                                                     'sudo kubeadm token create',
                                                     "openssl x509 -pubkey -in /etc/kubernetes/pki/ca.crt | openssl rsa -pubin -outform der 2>/dev/null | openssl dgst -sha256 -hex | sed 's/^.* //'"
                                                    ], 
                                       func_list=[check_in_while_loop,
                                                  exec_command_func,
                                                  exec_command_func
                                                 ])
        return result_list
    except Exception as e:
        return try_until_finish(master_ip)

# 나중에는 여러개 노드를 써서 master node의 high availability를 충족해야 겠지만, 아직은.

In [16]:

for i in range(len(master_list)):

    """
    순서대로
    1. master 동작 완료 체크
    2. token 생성 (찾는거보다 생성이 편함... 혼자쓸때만인가?)
    3. --discovery-token-ca-cert-hash 찾기
    """
    
    print(i,'th job start')

    master_ip = master_list[i].public_ip_address
    result_list = try_until_finish(master_ip)
    """
    result_list = exec_process(instance_ip=master_ip, 
                                   command_list=['sudo tail -n 1 /var/log/user-data.log',
                                                 'sudo kubeadm token create',
                                                 "openssl x509 -pubkey -in /etc/kubernetes/pki/ca.crt | openssl rsa -pubin -outform der 2>/dev/null | openssl dgst -sha256 -hex | sed 's/^.* //'"
                                                ], 
                                   func_list=[check_in_while_loop,
                                              exec_command_func,
                                              exec_command_func
                                             ])
    """

0 th job start
52.90.68.113
Get:8 https://download.docker.com/linux/ubuntu jammy/stable amd64 docker-ce amd64 5:24.0.1-1~ubuntu.22.04~jammy [22.8 MB]
Unpacking docker-ce (5:24.0.1-1~ubuntu.22.04~jammy) ...
No VM guests are running outdated hypervisor (qemu) binaries on this host.
Reading state information...
Unpacking kubectl (1.21.7-00) ...
[preflight] You can also perform this action in beforehand using 'kubeadm config images pull'
[preflight] You can also perform this action in beforehand using 'kubeadm config images pull'
[wait-control-plane] Waiting for the kubelet to boot up the control plane as static Pods from directory "/etc/kubernetes/manifests". This can take up to 4m0s
+ kubectl apply -f https://raw.githubusercontent.com/coreos/flannel/master/Documentation/kube-flannel.yml
kubernetes_ready
52.90.68.113 is ready
l9jocl.0pm8bcoaydhchsc4
64b485d37b401dd3d184d0b123e854295102f512828bf74b12d1bc690e528ecf


In [17]:
result_list

[True,
 'l9jocl.0pm8bcoaydhchsc4',
 '64b485d37b401dd3d184d0b123e854295102f512828bf74b12d1bc690e528ecf']

## node 연결 명령어 작성

In [18]:
master_private = master_list[0].private_ip_address
token = result_list[1]
discovery_token_ca_cert_hash = result_list[2]

join_command = """sudo kubeadm join {}:6443 --token {} --discovery-token-ca-cert-hash sha256:{}""".format(master_private,
                                                                                                token,
                                                                                                discovery_token_ca_cert_hash
                                                                                               )

join_command

'sudo kubeadm join 10.1.0.177:6443 --token l9jocl.0pm8bcoaydhchsc4 --discovery-token-ca-cert-hash sha256:64b485d37b401dd3d184d0b123e854295102f512828bf74b12d1bc690e528ecf'

## worker 연결 : 수가 많아지면 multi thread를 써야 겠지만 아직은.

In [19]:
for i in range(len(worker_list)):
    
    print(i,'th job start')

    worker_ip = worker_list[i].public_ip_address
    result_list = exec_process(instance_ip=worker_ip, 
                               command_list=['sudo tail -n 1 /var/log/user-data.log',
                                             join_command
                                            ], 
                               func_list=[check_in_while_loop,
                                          exec_command_func
                                         ])

0 th job start
174.129.95.87
kubernetes_ready
52.90.68.113 is ready
[preflight] Running pre-flight checks
[preflight] Reading configuration from the cluster...
[preflight] FYI: You can look at this config file with 'kubectl -n kube-system get cm kubeadm-config -o yaml'
[kubelet-start] Writing kubelet configuration to file "/var/lib/kubelet/config.yaml"
[kubelet-start] Writing kubelet environment file with flags to file "/var/lib/kubelet/kubeadm-flags.env"
[kubelet-start] Starting the kubelet
[kubelet-start] Waiting for the kubelet to perform the TLS Bootstrap...

This node has joined the cluster:
* Certificate signing request was sent to apiserver and a response was received.
* The Kubelet was informed of the new secure connection details.

Run 'kubectl get nodes' on the control-plane to see this node join the cluster.

1 th job start
3.92.163.138
kubernetes_ready
52.90.68.113 is ready
[preflight] Running pre-flight checks
[preflight] Reading configuration from the cluster...
[prefligh

In [20]:
for worker in worker_list:
    print(worker, worker.public_ip_address, worker.private_ip_address)

ec2.Instance(id='i-0780cdbb01b12ae6a') 174.129.95.87 10.1.0.233
ec2.Instance(id='i-0d046300b4951283c') 3.92.163.138 10.1.0.23


## 결과 확인 : 여러개 하니까 나중에 넣은건 한동안 not ready 뜨기도 하네, 시간지나니까 health check끝나서 연결됨

In [22]:

    master_ip = master_list[0].public_ip_address
    result_list = exec_process(instance_ip=master_ip, 
                               command_list=["sudo kubectl get nodes"], 
                               func_list=[exec_command_func])

52.90.68.113
NAME            STATUS   ROLES                  AGE   VERSION
ip-10-1-0-177   Ready    control-plane,master   80s   v1.21.7
ip-10-1-0-23    Ready    <none>                 47s   v1.21.7
ip-10-1-0-233   Ready    <none>                 57s   v1.21.7
